In [1]:
import pandas as pd
from datasets import load_dataset

import string
from nltk.stem.snowball import stopwords

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import numpy as np

from string import punctuation, ascii_lowercase, digits


import re
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

from datasets import load_dataset
from sklearn.model_selection import train_test_split


import fasttext
import tempfile
import os

from sklearn.metrics import classification_report
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import pickle

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Conv1D, SpatialDropout1D, Bidirectional
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [2]:
# Load Dataset
with open('split_data_v2.pkl', 'rb') as f:
    X_train, X_val, y_train, y_val = pickle.load(f)

In [3]:
X_train_all_list = (X_train['question'] + " " + X_train['best_answer']).tolist()
X_val_all_list = (X_val['question'] + " " + X_val['best_answer']).tolist()

In [7]:
# X_train_all = X_train.copy()
# X_val_all = X_val.copy()

# X_train_all['question_answer'] = X_train_all['question'] + " " + X_train_all['best_answer']
# X_val_all['question_answer'] = X_val_all['question'] + " " + X_val_all['best_answer']

In [8]:
# # save train and validation into txt format
# X_train_all.to_csv("FastText_all_emd_no_url.train",columns=["question_answer"],index=False,header=False)
# X_val_all.to_csv("FastText_all_emd_no_url.val",columns=["question_answer"],index=False,header=False)

In [6]:
# X_train_all = X_train_all[['question_answer']]
# X_val_all = X_val_all[['question_answer']]


In [9]:
# ft_model = fasttext.train_unsupervised("FastText_all_emd_no_url.train", model="skipgram", dim=100)

Read 53M words
Number of words:  182856
Number of labels: 0
Progress: 100.0% words/sec/thread:   93063 lr:  0.000000 avg.loss:  0.689027 ETA:   0h 0m 0s


In [11]:
# ft_model.save_model("ft_model_dim100.bin")

In [3]:
ft_model = fasttext.load_model("ft_model_dim100.bin")

In [4]:
ft_model.get_dimension()

100

In [6]:
def get_fasttext_embeddings_batch(texts, ft_model, max_length=100):
    embedding_dim = ft_model.get_dimension()  # FastText embedding dimension
    batch_embeddings = np.zeros((len(texts), max_length, embedding_dim), dtype=np.float32)

    for i, text in enumerate(texts):
        words = text.split()[:max_length]  # Truncate to max_length
        word_embeddings = [ft_model.get_word_vector(word) for word in words]
        batch_embeddings[i, :len(word_embeddings)] = word_embeddings  # Fill in the embeddings

    return batch_embeddings


def process_in_batches(texts, ft_model, batch_size=1000, max_length=100):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = get_fasttext_embeddings_batch(batch_texts, ft_model, max_length=max_length)
        all_embeddings.append(batch_embeddings)

    return np.vstack(all_embeddings)



In [7]:

max_length = 100  # Define the maximum sequence length
batch_size = 100  # Define the batch size

# Process training and validation embeddings in batches
X_train_emb = process_in_batches(X_train_all_list, ft_model, batch_size=batch_size, max_length=max_length)
X_val_emb = process_in_batches(X_val_all_list, ft_model, batch_size=batch_size, max_length=max_length)

print(f"Training Embeddings Shape: {X_train_emb.shape}")
print(f"Validation Embeddings Shape: {X_val_emb.shape}")

Training Embeddings Shape: (1085280, 100, 100)
Validation Embeddings Shape: (271320, 100, 100)


In [10]:
# np.save('X_train_emb.npy', X_train_emb)
# np.save('X_val_emb.npy', X_val_emb)

In [3]:
# Load training and validation embeddings
X_train_emb = np.load('X_train_emb.npy')
X_val_emb = np.load('X_val_emb.npy')

In [4]:
print(f"Training Embeddings Shape: {X_train_emb.shape}")
print(f"Validation Embeddings Shape: {X_val_emb.shape}")

Training Embeddings Shape: (1085280, 100, 100)
Validation Embeddings Shape: (271320, 100, 100)


In [11]:
# X_train_all_list = X_train_all['question_answer'].tolist()
# X_val_all_list = X_val_all['question_answer'].tolist()

In [8]:
# # Prepare training and validation embeddings
# max_length = 100  # Define the maximum sequence length
# X_train_emb = get_fasttext_embeddings(X_train_all_list, ft_model, max_length=max_length)
# X_val_emb = get_fasttext_embeddings(X_val_all_list, ft_model, max_length=max_length)

In [10]:
# X_train_emb.shape

(1, 100, 300)

In [9]:
# Build the Model
max_length = 100
sequence_input = Input(shape=(max_length, ft_model.get_dimension()), dtype='float32')

# Add layers with the advanced techniques
x = SpatialDropout1D(0.2)(sequence_input)
x = Conv1D(64, 5, activation='relu')(x)  # Convolution for local patterns
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)  # Bidirectional LSTM
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(512, activation='relu')(x)  # Another dense layer
outputs = Dense(len(set(y_train)), activation='softmax')(x)  # Output for multiclass classification

model = Model(sequence_input, outputs)


In [11]:
# Compile the Model
LR = 1e-3  # Learning rate
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (you can use 'val_accuracy' or others)
    patience=2,          # Number of epochs with no improvement to wait
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored metric
    verbose=1            # Display logs when training stops
)

# Define ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.1,          # Reduce learning rate by this factor
    patience=3,          # Number of epochs with no improvement before reducing learning rate
    min_lr=0.00001,      # Minimum learning rate
    verbose=1            # Display logs when learning rate is reduced
)


In [15]:
# Train the model with callbacks
history = model.fit(
    X_train_emb, y_train,
    batch_size=32,
    epochs=10,  # Set a high number, as training will stop early if no improvement
    validation_data=(X_val_emb, y_val),
    callbacks=[early_stopping, reduce_lr]  # Add both callbacks here
)

Epoch 1/10
33914/33915 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6651 - loss: 1.0576

: 

In [ ]:
# Visualize Training History
s, (at, al) = plt.subplots(2, 1, figsize=(10, 10))

# Accuracy plot
at.plot(history.history['accuracy'], c='b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('Model Accuracy')
at.set_ylabel('Accuracy')
at.set_xlabel('Epoch')
at.legend(['Train', 'Validation'], loc='upper left')

# Loss plot
al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('Model Loss')
al.set_ylabel('Loss')
al.set_xlabel('Epoch')
al.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [5]:
import sys
import psutil
import os
import gc

def get_memory_usage():
    """
    Check current memory usage and available memory of the Python process.
    """
    process = psutil.Process(os.getpid())
    total_mem = psutil.virtual_memory().total / (1024 ** 3)  # Total memory in GB
    available_mem = psutil.virtual_memory().available / (1024 ** 3)  # Available memory in GB
    used_mem = process.memory_info().rss / (1024 ** 3)  # Memory used by this Python process in GB

    print(f"Total Memory: {total_mem:.2f} GB")
    print(f"Available Memory: {available_mem:.2f} GB")
    print(f"Memory Used by Process: {used_mem:.2f} GB")

def list_large_variables(threshold=10):
    """
    List variables in the global scope that exceed a memory threshold.
    Args:
        threshold (int): Memory size threshold in MB to list variables.
    """
    large_vars = []
    for var_name, var_value in globals().items():
        try:
            size = sys.getsizeof(var_value) / (1024 ** 2)  # Convert size to MB
            if size > threshold:
                large_vars.append((var_name, size))
        except Exception as e:
            pass  # Skip objects that can't be sized

    if large_vars:
        print("Large variables (size > {} MB):".format(threshold))
        for name, size in sorted(large_vars, key=lambda x: -x[1]):
            print(f"{name}: {size:.2f} MB")
    else:
        print("No large variables found exceeding the threshold.")

# Example usage
get_memory_usage()  # Check current memory usage and available memory
list_large_variables(threshold=10)  # List variables larger than 10 MB

Total Memory: 16.00 GB
Available Memory: 8.90 GB
Memory Used by Process: 2.19 GB
Large variables (size > 10 MB):
X_train: 448.35 MB
X_val: 111.99 MB
y_train: 16.56 MB
